In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력합니다:

print(device)

cuda:0


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

In [ ]:
trainset2 = iter(trainset)
images, labels = next(trainset2)
print(len(trainset), images.shape, labels)
print(trainset[0][0].shape)

print(len(trainloader))

for idx, data in enumerate(trainloader):
  print(data[1][0])
  data[1][0] = -111
  print(data[1][0])
  break
for idx, data in enumerate(trainloader):
  print(data[1])
  break
for images, labels in trainloader:
  print(len(trainloader), images.shape, labels)
  break
for images, labels in trainloader:
  pass
print(images.shape, labels.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchsummary

In [ ]:

# output = input-kernel+(2*padding) / stride + 1
# output = 32 - 3 + (2) / 1 + 1

class AlexNet(nn.Module) :
    def __init__(self) :
        super(AlexNet, self).__init__()
        self.name = "AlexNet"
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=(4, 4)),
            nn.ReLU(inplace=True),
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=(5, 5), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.fc_layer1 = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(9216, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10)
        )

    def forward(self, x) :
        output = self.conv_layer1(x)
        output = self.conv_layer2(output)
        output = self.conv_layer3(output)
        output = output.view(-1, 9216)
        output = self.fc_layer1(output)
        return output

In [ ]:
model = AlexNet()
model = model.to(device)

In [ ]:
torchsummary.summary(model, input_size=(3, 32, 32), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           2,688
              ReLU-2           [-1, 96, 32, 32]               0
         MaxPool2d-3           [-1, 96, 30, 30]               0
            Conv2d-4          [-1, 256, 30, 30]         221,440
              ReLU-5          [-1, 256, 30, 30]               0
         MaxPool2d-6          [-1, 256, 15, 15]               0
            Conv2d-7          [-1, 256, 15, 15]         590,080
              ReLU-8          [-1, 256, 15, 15]               0
         MaxPool2d-9            [-1, 256, 7, 7]               0
          Dropout-10                [-1, 12544]               0
           Linear-11                 [-1, 2048]      25,692,160
             ReLU-12                 [-1, 2048]               0
           Linear-13                   [-1, 10]          20,490
Total params: 26,526,858
Trainable para

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.train()
for epoch in range(10):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;


        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()
        # one hot encoding
        inputs, labels = data

        inputs = inputs.to(device)
        labels = torch.Tensor(np.eye(10)[labels]).to(device)
        
        # 순전파 + 역전파 + 최적화를 한 후
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 30 == 29:
            # print(outputs.shape, labels.shape)
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 30:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    30] loss: 100.141
[1,    60] loss: 2.109
[1,    90] loss: 1.983
[1,   120] loss: 1.884
[1,   150] loss: 1.847
[1,   180] loss: 1.837
[2,    30] loss: 1.765
[2,    60] loss: 1.731
[2,    90] loss: 1.698
[2,   120] loss: 1.713
[2,   150] loss: 1.701
[2,   180] loss: 1.657
[3,    30] loss: 1.634
[3,    60] loss: 1.638
[3,    90] loss: 1.618
[3,   120] loss: 1.608
[3,   150] loss: 1.620
[3,   180] loss: 1.601
[4,    30] loss: 1.598
[4,    60] loss: 1.583
[4,    90] loss: 1.565
[4,   120] loss: 1.584
[4,   150] loss: 1.581
[4,   180] loss: 1.559
[5,    30] loss: 1.529
[5,    60] loss: 1.547
[5,    90] loss: 1.555
[5,   120] loss: 1.542
[5,   150] loss: 1.559
[5,   180] loss: 1.561
[6,    30] loss: 1.509
[6,    60] loss: 1.503
[6,    90] loss: 1.553
[6,   120] loss: 1.569
[6,   150] loss: 1.508
[6,   180] loss: 1.518
[7,    30] loss: 1.504
[7,    60] loss: 1.489
[7,    90] loss: 1.532
[7,   120] loss: 1.514
[7,   150] loss: 1.492
[7,   180] loss: 1.522
[8,    30] loss: 1.481
[8,    60

In [ ]:
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = model(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 51 %


In [ ]:
# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

In [ ]:
import matplotlib.pyplot as plt

with torch.no_grad():
    
    model.to('cpu')
    model.eval()
    y_pred = model(images)
    y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
    y = labels.detach().numpy()
    

In [ ]:
def display_test(x, y, y_pred):
    plt_w = 10
    plt_h = int(np.round(x.shape[0] / plt_w))

    plt.figure(figsize=(18,18))

    for y_pos in range(plt_h):
        for x_pos in range(plt_w):
            idx = y_pos * plt_w + x_pos
            if idx >= x.shape[0] :
                break

            plt.subplot(plt_w, plt_h, idx+1)

            prt = '[%s, %s]' % (classes[y[idx]], classes[y_pred[idx]])
            plt.title(prt)
            plt.imshow(x[idx])

            ax = plt.gca()
            ax.axes.xaxis.set_visible(False)
            ax.axes.yaxis.set_visible(False)

    plt.show()

In [ ]:
images = images.numpy().transpose(0, 2, 3, 1) / 2 + 0.5 

In [ ]:
display_test(images[:60], y[:60], y_pred[:60])